In [1]:
import numpy as np
import scipy as sp

# Define Hamiltonian
We want a single body Hamiltonian as a $d \times d$ matrix. We can either choose a specific Hamiltonian (TFI, AKLT, cluster state) or a random Hamiltonian.

In [2]:
# Random Hamiltonian
d = 32
H = np.random.rand(d,d) + 1.j * np.random.rand(d,d)
H = H + H.conj().T

In [4]:
# Periodic TFI
def kron(ops):
    op = 1
    for o in ops:
        op = np.kron(op, o)
    return op

Z = np.array([[1,0],[0,-1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1.j],[1.j,0]])
Id = np.eye(2)

def build_TFI_Hamiltoinan(L, Jz, hx, hz):
    # Build full 2^L x 2^L Hamiltonian

    H = np.zeros((2**L, 2**L), dtype=np.float64)

    for i in range(L):
        H += hx * kron([Id]*i + [X] + [Id]*(L-i-1))
        H += hz * kron([Id]*i + [Z] + [Id]*(L-i-1))

    Ising = [Z, Z] + [Id] * (L-2)
    for i in range(L):
        H += Jz * kron(Ising)
        # Shift the first element to the last; periodic BCs!
        Ising.append(Ising.pop(0))
    return H
    
def build_cluster_state_Hamiltoinan(L):
    # Build full 2^L x 2^L Hamiltonian

    H = np.zeros((2**L, 2**L), dtype=np.float64)

    Cluster = [Z, X, Z] + [Id] * (L-3)
    for i in range(L):
        H += -1 * kron(Cluster)
        # Shift the first element to the last; periodic BCs!
        Cluster.append(Cluster.pop(0))
    return H

#H = build_TFI_Hamiltoinan(5, 1, -1.4, 0.9045)
H = build_cluster_state_Hamiltoinan(5)
d = H.shape[0]

In [5]:
L, U = np.linalg.eigh(H)
assert np.isclose(np.linalg.norm(U @ np.diag(L) @ U.conj().T - H), 0.0)

In [6]:
print(L)

[-5. -3. -3. -3. -3. -3. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  3.  3.  3.  3.  3.  5.]


# Define Kraus operators for discrete quantum channel

In [7]:
L_shift = L - np.min(L)
print(L_shift)
H_shift = U @ np.diag(L_shift) @ U.conj().T

[ 0.  2.  2.  2.  2.  2.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  8.  8.  8.  8.  8. 10.]


In [8]:
beta = 0.1
gamma_1 = sp.linalg.expm(-beta * H_shift)
assert np.isclose(np.linalg.norm(U @ sp.linalg.expm(-beta * np.diag(L_shift)) @ U.conj().T - gamma_1), 0.0)

In [9]:
gamma_2 = U @ np.diag(np.sqrt(1 - np.exp(-2*beta*L_shift))) @ U.conj().T

In [10]:
P = np.roll(np.eye(d), 1, axis=1)
P_gamma_2 = P @ gamma_2

In [11]:
assert np.isclose(np.linalg.norm(np.eye(d) - (gamma_1.conj().T @ gamma_1 + gamma_2.conj().T @ gamma_2)), 0.0)
assert np.isclose(np.linalg.norm(np.eye(d) - (gamma_1.conj().T @ gamma_1 + P_gamma_2.conj().T @ P_gamma_2)), 0.0)

In [12]:
assert np.isclose(np.linalg.norm(gamma_1 - gamma_1.conj().T), 0.0)
assert np.isclose(np.linalg.norm(gamma_2 - gamma_2.conj().T), 0.0)
assert not np.isclose(np.linalg.norm(P_gamma_2 - P_gamma_2.conj().T), 0.0)

Both $\Gamma_1$ and $\Gamma_2$ are Hermitian, as they should be. Both are matrix exponentials of Hermitian matrices with a real exponent (imaginary time evolution and all). Additionally, we have checked that $\sum_i K_i^\dagger K_i = 1$. $P \Gamma_2$ is no longer Hermitian since we have applied a permutation matrix.

# Define vectorized channel
We want the superoperator $E = \sum_i K_i \otimes \overline{K_i}$, given Kraus operators $K_i$.

In [13]:
E = np.kron(gamma_1, gamma_1.conj()) + np.kron(gamma_2, gamma_2.conj())
print(E.shape)

(1024, 1024)


In [14]:
P_E = np.kron(gamma_1, gamma_1.conj()) + np.kron(P_gamma_2, P_gamma_2.conj())
print(P_E.shape)

(1024, 1024)


In [15]:
assert np.isclose(np.linalg.norm(E - E.conj().T), 0.0)
assert not np.isclose(np.linalg.norm(P_E - P_E.conj().T), 0.0)

The superoperator without permutation is Hermitian. This makes sense since all Kraus operators are. However, the superoperator isn't.

In [16]:
L2, U2 = np.linalg.eigh(E)
assert np.isclose(np.linalg.norm(E - U2 @ np.diag(L2) @ U2.conj().T), 0.0)
#assert np.isclose(np.linalg.norm(E - U2 @ np.diag(L2) @ sp.linalg.inv(U2)), 0.0)

Columns of $U2$ are the right eigenvectors. I am using `np.linalg.eig` rather than `np.linalg.eigh` since the fixed point eigenvectors for `eigh` aren't Hermitian when viewed as density matrices. I am not sure why this is the case.

In [17]:
print(np.max(L2))
L2_index = np.where(np.isclose(np.abs(L2), 1.0, atol=1.e-14, rtol=1.e-14))[0]
print(L2_index, len(L2_index))

1.0000000000000018
[ 772  773  774  775  776  777  778  779  780  781  782  783  784  785
  786  787  788  789  790  791  792  793  794  795  796  797  798  799
  800  801  802  803  804  805  806  807  808  809  810  811  812  813
  814  815  816  817  818  819  820  821  822  823  824  825  826  827
  828  829  830  831  832  833  834  835  836  837  838  839  840  841
  842  843  844  845  846  847  848  849  850  851  852  853  854  855
  856  857  858  859  860  861  862  863  864  865  866  867  868  869
  870  871  872  873  874  875  876  877  878  879  880  881  882  883
  884  885  886  887  888  889  890  891  892  893  894  895  896  897
  898  899  900  901  902  903  904  905  906  907  908  909  910  911
  912  913  914  915  916  917  918  919  920  921  922  923  924  925
  926  927  928  929  930  931  932  933  934  935  936  937  938  939
  940  941  942  943  944  945  946  947  948  949  950  951  952  953
  954  955  956  957  958  959  960  961  962  963  964  9

In [18]:
L2_sort = np.abs(L2)
L2_sort.sort()
print(L2_sort[-len(L2_index)-1:])

[0.9960166 1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.        1.       

Why are there multiple ($d$ for random Hamiltonian) fixed points? I guess there isn't a unique fixed point. This is because we haven't included the permutation. Need to understand why there are $d$ fixed points. I should look into purity and energies of each of the fixed points.

In [19]:
L3, U3 = np.linalg.eig(P_E)
assert np.isclose(np.linalg.norm(P_E - U3 @ np.diag(L3) @ sp.linalg.inv(U3)), 0.0, atol=1.e-7, rtol=1.e-7)

For the non-Hermitian matrices, we need to use the inverse rather than the Hermitian conjugate.

In [20]:
print(np.max(L3))
L3_index = np.where(np.isclose(np.abs(L3), 1.0, atol=1.e-14, rtol=1.e-14))[0]
print(L3_index, len(L3_index))

(0.9999999999999906+0j)
[2] 1


Now, with the permutation inserted, we have a unique fixed point (at least for our random Hamiltonian)! The permutation is doing something.

In [21]:
L3_sort = np.abs(L3)
L3_sort.sort()
print(L3_sort[-len(L3_index)-1:])

[0.98458839 1.        ]


We have a spectral gap!

# Investigate fixed points

## Permutation channel

In [22]:
L_fp = []
valid_fps = []
for i in L3_index:
    L_fp.append(L3[i])
    fp = U3[:,i].reshape(d,d)
    assert np.isclose(np.linalg.norm(fp - fp.conj().T), 0.0)
    fp = fp + fp.conj().T
    fp /= fp.trace()
    print(i, L3[i], np.linalg.eigh(fp)[0])
    valid_fps.append(fp)

2 (0.9999999999999906+0j) [-3.78158781e-15 -3.23451335e-15 -3.02599809e-15 -2.82658780e-15
 -2.70900663e-15 -2.32314236e-15 -2.25729485e-15 -2.10241779e-15
 -1.92451579e-15 -1.90344448e-15 -1.59479696e-15 -1.58754621e-15
 -1.55640739e-15 -1.35204581e-15 -1.23238473e-15 -1.18103314e-15
 -1.00568001e-15 -9.51338467e-16 -8.27404369e-16 -6.38484980e-16
 -5.77303298e-16 -5.67207671e-16 -4.18340489e-16 -4.10634711e-16
 -2.40239598e-16 -2.19327419e-16 -1.99171850e-16 -5.97520032e-17
  5.56364662e-17  1.87419786e-16  3.97605779e-16  1.00000000e+00]


In [23]:
print("GROUND STATE ENERGY:", L[0])
for i, vfp in enumerate(valid_fps):
    print("Fixed point:", i)
    print("Purity:", np.trace(vfp @ vfp))
    print("Energy:", np.trace(H @ vfp))

GROUND STATE ENERGY: -4.9999999999999964
Fixed point: 0
Purity: (1.000000000000078+0j)
Energy: (-5.000000000000172+0j)


For the permutation channel, the single fixed point is the ground state; it's a pure quantum state and has the ground state energy.

## Non-permutation channel
The eigenvectors form a basis for the fixed point subspace, but they themselves are not Hermitian or PSD.

In [24]:
L_fp = []
valid_fps = []
for i in L2_index:
    L_fp.append(L2[i])
    fp = U2[:,i].reshape(d,d)
    if np.linalg.norm(fp - fp.conj().T) < 1.e-4:
    #assert np.isclose(np.linalg.norm(fp - fp.conj().T), 0.0)
        fp = fp + fp.conj().T
        fp /= fp.trace()
        print(i, L2[i], np.linalg.eigh(fp)[0])
        valid_fps.append(fp)

923 0.9999999999999999 [-8.50226158e-15 -8.22404897e-15 -7.30853430e-15 -5.66344774e-15
 -5.56886840e-15 -3.69102837e-15 -3.33423680e-15 -2.03054680e-15
 -1.13704853e-15 -1.05930790e-15 -9.15118205e-16 -4.67050463e-16
 -1.56320244e-16  4.76615417e-17  4.82890505e-16  8.41569345e-16
  1.48348596e-15  1.48493412e-15  2.34810826e-15  2.79050955e-15
  3.14111279e-15  3.34557194e-15  4.97549139e-15  5.06780619e-15
  5.86485524e-15  7.75555204e-15  3.12500000e-02  3.12500000e-02
  1.56250000e-01  1.56250000e-01  3.12500000e-01  3.12500000e-01]


In [25]:
print("GROUND STATE ENERGY:", L[0])
for i, vfp in enumerate(valid_fps):
    print("Fixed point:", i)
    print("Purity:", np.trace(vfp @ vfp))
    print("Energy:", np.trace(H @ vfp))

GROUND STATE ENERGY: -4.9999999999999964
Fixed point: 0
Purity: 0.24609375000000205
Energy: -4.1330939549745036e-14
